In [ ]:
%reload_ext autoreload
%autoreload 2

In [5]:
from dotenv import load_dotenv
import hopsworks
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.load_transform_data import transform_ohlc_to_features_target, get_new_ethereum_ohlc, get_new_ethereum_ohlc_2

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


### Download the last 30 days of Ethereum OHLC data.

In [6]:
##### This is the scraping functino that is used in github actions - it wont work locally unless you have chromeDriver installed #####
# But if you want ChromeDriver locally, you can do this (Mac):
# 1. brew install chromedriver
# 2. xattr -d com.apple.quarantine $(which chromedriver)
# 3. which chromedriver
# 4. the above line give the path to chromedriver. Now you can go into src/load_transfrom_data.py, and change the path in the get_new_ethereum_ohlc() function.  
new_ethereum_ohlc = get_new_ethereum_ohlc()

##### Use this function when you running locally. This one doesn't require chromeDriver
# new_ethereum_ohlc = get_new_ethereum_ohlc_2()

### Update our historical Ethereum OHLC data with the newly retrieved data

In [40]:
historical_ethereum_ohlc = pd.read_csv('../data/raw/historical_ohlc/ethereum.csv', parse_dates=['Date'])
historical_ethereum_ohlc['Date'] = pd.to_datetime(historical_ethereum_ohlc['Date'])
new_ethereum_ohlc['Date'] = pd.to_datetime(new_ethereum_ohlc['Date'])
combined_df = pd.concat([new_ethereum_ohlc, historical_ethereum_ohlc], ignore_index=True)
combined_df = combined_df.drop_duplicates(subset='Date', keep='last')

# Update our historical data with new observations
combined_df.to_csv('../data/raw/historical_ohlc/ethereum.csv', index=False)

### Perform the transformation of raw data into predictors and response varaibles 

In [57]:
features_and_target = transform_ohlc_to_features_target()
features_and_target

,date,open,high,low,close,volume,volume_eth,market_cap,tommorow_price,target,...,sma_10,ema_25,rsi_25,sma_25,ema_50,rsi_50,sma_50,ema_100,rsi_100,sma_100
3054,2015-11-15,0.8912,0.9215,0.8750,0.9064,4.118000e+05,458365.0,6.720000e+07,0.9290,1,...,0.984672,1.040036,50.958458,1.024686,1.049078,46.417700,1.216422,0.942045,43.066211,0.959506
3053,2015-11-16,0.9062,0.9447,0.8920,0.9290,6.209000e+05,676442.0,6.860000e+07,1.0100,1,...,1.008894,1.060586,51.682152,1.033352,1.072073,46.821823,1.239824,0.966193,43.328367,0.981604
3052,2015-11-17,0.9249,1.0300,0.9058,1.0100,1.100000e+06,1183690.0,7.220000e+07,0.9900,0,...,1.087179,1.139641,54.205128,1.100424,1.158029,48.249270,1.332732,1.049388,44.257583,1.063728
3051,2015-11-18,0.9900,1.0100,0.9405,0.9900,6.811000e+05,691994.0,7.360000e+07,0.9555,0,...,1.070259,1.107104,53.486795,1.058955,1.129116,47.925148,1.295100,1.028025,44.077332,1.039581
3050,2015-11-19,0.9887,1.0100,0.9375,0.9555,4.435000e+05,455866.0,7.280000e+07,0.9249,0,...,1.037955,1.062920,52.242769,1.007418,1.085945,47.365082,1.242914,0.992353,43.766743,1.004561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2024-04-09,3700.0000,3720.0000,3466.0000,3506.0000,1.610000e+10,4465773.0,4.405000e+11,3543.0000,1,...,1.019186,1.009481,52.966019,1.011751,1.043049,55.496861,1.005587,1.144748,56.631814,1.182570
4,2024-04-10,3503.0000,3555.0000,3421.0000,3543.0000,1.580000e+10,4509418.0,4.292000e+11,3506.0000,0,...,1.033095,1.018557,53.698795,1.022204,1.051827,55.906399,1.013125,1.153248,56.884572,1.190260
3,2024-04-11,3542.0000,3613.0000,3486.0000,3506.0000,1.270000e+10,3574952.0,4.331000e+11,3242.0000,0,...,1.022247,1.007306,52.841241,1.013154,1.039178,55.386308,0.999498,1.138022,56.551649,1.173305
2,2024-04-12,3503.0000,3551.0000,3173.0000,3242.0000,1.950000e+10,5680677.0,4.202000e+11,3013.0000,0,...,0.946183,0.936394,47.234772,0.939939,0.962403,51.872853,0.922811,1.051240,54.262791,1.081222


In [65]:
print(features_and_target['date'].nunique())
print(features_and_target['close'].nunique())
print(features_and_target['open'].nunique())
print(features_and_target['ema_10'].nunique())
print(features_and_target['ema_2'].nunique())

3054
2675
2655
3054
3054


### Update our Hopsworks feature store with the updated data

In [58]:
load_dotenv('../.env')

HOPSWORKS_PROJECT_NAME = os.environ['HOPSWORKS_PROJECT_NAME']
HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
FEATURE_GROUP_NAME = 'ethereum_ohlc_feature_group'
FEATURE_GROUP_VERSION = 1

project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

feature_store = project.get_feature_store()

feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Historical daily ethereum OHLC time series",
    # primary_key = ['date'],
    # event_time='date',
)

feature_group.insert(features_and_target, write_options={"wait_for_job": False})

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 100.00% |██████████| Rows 3054/3054 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: ethereum_ohlc_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/582805/jobs/named/ethereum_ohlc_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1388f46d0>, None)